## Módulo 14 - K-Means

### Bloco 1 - Análise Exploratória de Dados

#### Setup e Carga de Dados

Neste vídeo, vamos começar a trabalhar com o algoritmo K-Means no contexto de clusterização de clientes. O objetivo é agrupar os clientes em clusters com base em suas características, para oferecer um atendimento personalizado. Vou criar um arquivo Jupyter Notebook chamado "clusterização-clientes.ipynb" e importar as bibliotecas necessárias, como Pandas, Plotly, Scikit-Learn, Optuna, entre outras. Também vamos utilizar algumas métricas específicas para avaliar os resultados da clusterização. Além disso, faremos a carga dos dados de clientes a partir de um arquivo CSV e faremos uma análise inicial da estrutura desses dados. No próximo vídeo, continuaremos com o desenvolvimento do projeto.

#### EDA

Neste vídeo, vamos dar continuidade ao nosso projeto no Jupyter Notebook, focando agora na Análise Exploratória de Dados (EDA). A ideia aqui não é repetir o EDA que já fizemos nos módulos anteriores, mas sim explorar algumas técnicas novas, como testes estatísticos. Começaremos analisando a distribuição da variável "inovação" em nosso conjunto de dados. Em seguida, levantaremos a hipótese de que empresas com diferentes níveis de inovação possam ter médias de faturamento mensal distintas. Para testar essa hipótese, utilizaremos o teste estatístico ANOVA (Análise de Variância), que verifica se há variações significativas nas médias de diferentes grupos. No entanto, antes de aplicar o teste, precisamos validar algumas suposições, como a independência das observações, a normalidade dos dados e a homogeneidade das variâncias. Caso essas suposições não sejam atendidas, utilizaremos o teste de ANOVA de Welch, que é mais robusto nessas situações. Nos próximos vídeos, veremos como realizar essas validações e aplicar o teste de hipótese.

#### Introdução ANOVA e Testes Estatísticos

Nesta aula, vamos analisar os pressupostos necessários para realizar um teste estatístico de ANOVA. Vamos verificar se as variâncias entre os grupos são homogêneas e se os dados seguem uma distribuição normal. Também vamos considerar que as observações são independentes e que a variável independente é contínua. Além disso, vamos observar que as amostras não têm tamanhos iguais. Para verificar a homogeneidade das variâncias, vamos utilizar o teste de Bartlett. Para verificar a distribuição normal, vamos utilizar o teste de Shapiro-Wilk. Com base nos resultados desses testes, poderemos determinar se os pressupostos foram atendidos e prosseguir com o teste de ANOVA adequado.

#### Teste de ANOVA

Neste vídeo, concluímos a parte de EDA (Exploratory Data Analysis) com um teste de ANOVA para avaliar a relação entre as médias dos valores para cada nível de inovação. Utilizamos o teste de ANOVA de Welch, pois as amostras têm tamanhos diferentes. Para isso, utilizamos o pacote estatístico PINGUIM e importamos o módulo Welch ANOVA. O resultado do teste mostrou que não há diferenças significativas entre as médias dos grupos, independentemente do nível de inovação das empresas. Isso significa que empresas com diferentes níveis de inovação podem ter faturamentos semelhantes. No próximo vídeo, exploraremos o algoritmo de K-Means.

### Bloco 2 - Treinamento do Modelo

#### Preparação dos Dados para o KMeans

Neste vídeo, começamos a trabalhar com o algoritmo de K-Means. Para treinar o algoritmo, precisamos preparar os dados. Faremos transformações nas colunas para que estejam no formato correto. Selecionaremos todas as colunas, pois não temos um target específico. Separaremos as variáveis numéricas, categóricas e ordinais. Aplicaremos transformações em cada tipo de variável usando um standard scaler, one-hot-encoder e ordinal encoder. Em seguida, criaremos um pré-processador que transformará os dados em um novo dataset. Usaremos o column transformer para indicar quais transformadores usar. Por fim, faremos o fit transform no pré-processador para treinar os transformadores e aplicar as transformações nos dados.

#### Executando o KMeans com Optuna

Neste vídeo, vamos preparar os dados para treinar nosso modelo de algoritmo. Vamos otimizar a métrica de silhueta usando o Optuna para ajustar os hiperparâmetros. Vamos definir uma função chamada K-Means Objective, que receberá uma tentativa de ajuste dos hiperparâmetros. Vamos ajustar dois hiperparâmetros: o número de clusters (K) e a métrica de distância (Euclidiana e Minkowski). Em seguida, vamos instanciar o K-Means com esses hiperparâmetros e treinar o modelo. Calcularemos o Silhouette Score usando a função pairwise distance e os rótulos do K-Means. Por fim, criaremos um estudo do Optuna e rodaremos o experimento para encontrar a melhor configuração de hiperparâmetros.

### Bloco 3 - Análise de Resutados

#### Analisando Resultados do KMeans

Neste vídeo, vamos analisar os resultados do Optuna e treinar um modelo com os melhores parâmetros encontrados. Primeiro, vamos guardar a melhor configuração em uma variável chamada "best_params". Em seguida, vamos criar o modelo usando esses melhores parâmetros e ajustá-lo aos dados. Calcularemos o score de silhueta para avaliar o desempenho do modelo. O Optuna determinou que o número de clusters ideal é 3, a métrica de distância escolhida foi a euclidiana e o score de silhueta foi de 0.44. Agora, vamos atribuir os clusters aos registros do nosso dataset e visualizar os primeiros registros com a coluna de cluster atualizada. No próximo vídeo, iremos explorar esses clusters em uma visualização de dados.

#### Visualização Decisões do KMeans

Neste vídeo, vamos analisar os clusters gerados pelo algoritmo e tentar entender os critérios utilizados para organizá-los. Faremos cruzamentos de dados, como idade e faturamento mensal, e criaremos visualizações para identificar padrões. Observamos que não há um padrão claro baseado no faturamento, mas encontramos um padrão interessante relacionado à inovação. Empresas com níveis de inovação baixos foram agrupadas em um cluster, independentemente do faturamento, enquanto empresas com níveis de inovação moderados e avançados foram agrupadas em outros clusters. Isso nos revela a possibilidade de personalizar o atendimento aos clientes com base em seu nível de inovação. Salvamos o modelo e o pipeline para uso futuro na aplicação batch.

### Bloco 4 - Entrega do Modelo com App Batch

#### Aplicação Batch com Gradio

Nesta aula, desenvolvemos uma aplicação simples no Gradle. Utilizamos um algoritmo de clusterização não supervisionado para transformar os dados em um pipeline separado. Não fizemos predições, apenas trouxemos uma estrutura com o algoritmo. Carregamos o modelo, definimos uma função chamada "clustering" que recebe um arquivo CSV, lê o arquivo, transforma os dados para o formato necessário, treina o modelo e adiciona a coluna de cluster no DataFrame. Salvamos o DataFrame em um arquivo CSV e retornamos o nome desse arquivo. Criamos uma interface para chamar a função e rodamos a aplicação. Agora podemos fazer upload de um arquivo, que será processado e gerará um arquivo de saída com os clusters.